In [7]:
from typing import Optional
from orchestrator.ingestion.vector_store.pinecone.client import PineconeClient
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    Event,
    step,
)
from orchestrator.ingestion.utils.pdf_processor import process_pdfs_in_directory


In [8]:
# events.py
class InitializeEvent(Event):
    pass

class ConciergeEvent(Event):
    request: Optional[str]
    just_completed: Optional[str]
    need_help: Optional[bool]

class OrchestratorEvent(Event):
    request: Optional[str] = None

class PDFIngestionEvent(Event):
    request: Optional[str]


In [9]:
class WorkflowSteps:
    
    @step(pass_context=True)
    async def concierge(self, ctx: Context, ev: ConciergeEvent | StartEvent) -> InitializeEvent | StopEvent | OrchestratorEvent | None:
        if isinstance(ev, StartEvent):
            dirname = ev.get("dirname")
            if dirname is not None:
                ctx.data["dirname"] = dirname
                return OrchestratorEvent(request="ingest")  # Trigger the OrchestratorEvent for ingestion

            return InitializeEvent()

        return StopEvent(result={"index": ctx.data["index"]})

    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> ConciergeEvent:
        ctx.data["index"] = PineconeClient(collection_name="pdf-docs")
        return ConciergeEvent()

    @step(pass_context=True)
    async def orchestrator(self, ctx: Context, ev: OrchestratorEvent) -> ConciergeEvent | PDFIngestionEvent | StopEvent:
        if ev.request == "ingest":
            return PDFIngestionEvent(request="start_pdf_ingestion")
        return StopEvent(result={"message": "Orchestrator did not recognize the event"})

    @step(pass_context=True)
    async def ingest(self, ctx: Context, ev: PDFIngestionEvent) -> InitializeEvent | None:
        dirname = ctx.data.get("dirname")
        if dirname:
            nodes = process_pdfs_in_directory(dirname)

            client = PineconeClient(collection_name="pdf-docs")
            client.upsert_indices(nodes)
            ctx.data["dirname"] = None
            return InitializeEvent()
        return None


In [10]:
# workflow.py

class ConciergeWorkflow(Workflow):
    
    steps = WorkflowSteps()

    @step(pass_context=True)
    async def concierge(self, ctx: Context, ev: ConciergeEvent | StartEvent) -> InitializeEvent | StopEvent | OrchestratorEvent | None:
        return await self.steps.concierge(ctx, ev)

    @step(pass_context=True)
    async def initialize(self, ctx: Context, ev: InitializeEvent) -> ConciergeEvent:
        return await self.steps.initialize(ctx, ev)

    @step(pass_context=True)
    async def orchestrator(self, ctx: Context, ev: OrchestratorEvent) -> ConciergeEvent | PDFIngestionEvent | StopEvent:
        return await self.steps.orchestrator(ctx, ev)

    @step(pass_context=True)
    async def ingest(self, ctx: Context, ev: PDFIngestionEvent) -> InitializeEvent | None:
        return await self.steps.ingest(ctx, ev)


In [11]:
# main.py

from workflow import ConciergeWorkflow
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(ConciergeWorkflow, filename="concierge_flows.html")

concierge = ConciergeWorkflow(timeout=180, verbose=True)
result = await concierge.run()
query_engine = result["index"].index.as_query_engine()
response = query_engine.query("how does Environmental difficulties at present are also challenging the sustainability of the Mediterranean way of living")
print(str(result))


concierge_flows.html
Running step concierge
Step concierge produced event InitializeEvent
Running step initialize
Step initialize produced event ConciergeEvent
Running step concierge
Step concierge produced event StopEvent
{'index': <ingestion.vector_store.pinecone.client.PineconeClient object at 0x13f3c8c10>}


In [12]:
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

Environmental difficulties at present are challenging the sustainability of the Mediterranean way of living through factors such as water scarcity, land degradation, and declining biodiversity. These challenges are exacerbated by issues like urbanization, pollution, soil erosion, and climate change, which impact water resources, agricultural productivity, and the overall ecosystem health in the Mediterranean regions.